In [1]:
# MNIST Neural Network
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import random

In [2]:
device = 'cpu'

random.seed(777)
torch.manual_seed(777)

learning_rate = 0.001
training_epochs = 30
batch_size = 100

In [3]:
mnist_train = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=False)

mnist_test = dsets.MNIST(root='/tmp/pycharm_project_001/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [4]:
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [5]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:

total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis =model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print('Epoch : ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning finished')

Epoch :  0001 cost =  129.347152710
Epoch :  0002 cost =  36.114086151
Epoch :  0003 cost =  23.035547256
Epoch :  0004 cost =  16.021694183
Epoch :  0005 cost =  11.682225227
Epoch :  0006 cost =  8.563262939
Epoch :  0007 cost =  6.367186546
Epoch :  0008 cost =  4.798729897
Epoch :  0009 cost =  3.599060297
Epoch :  0010 cost =  2.749149561
Epoch :  0011 cost =  2.028066635
Epoch :  0012 cost =  1.750852585
Epoch :  0013 cost =  1.293702245
Epoch :  0014 cost =  0.984640956
Epoch :  0015 cost =  0.782710016
Epoch :  0016 cost =  0.620162666
Epoch :  0017 cost =  0.594548404
Epoch :  0018 cost =  0.576817870
Epoch :  0019 cost =  0.524664581
Epoch :  0020 cost =  0.417893976
Epoch :  0021 cost =  0.426523805
Epoch :  0022 cost =  0.371511370
Epoch :  0023 cost =  0.270034373
Epoch :  0024 cost =  0.306643367
Epoch :  0025 cost =  0.301861763
Epoch :  0026 cost =  0.326715648
Epoch :  0027 cost =  0.324042946
Epoch :  0028 cost =  0.254138947
Epoch :  0029 cost =  0.237113684
Epoch : 

In [7]:
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label : ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction : ', torch.argmax(single_prediction, 1).item())

Accuracy :  0.9557999968528748
Label :  8
Prediction :  3
